In [1]:
import tensorflow as tf # Tensorflow of ANN
import pandas as pd # Pandas DataFrame
import numpy as np # Numpy array manipulation
from sklearn.model_selection import KFold # k-fold cross validation
from sklearn.metrics import accuracy_score, f1_score # Evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer # Missing value imputation
from keras.utils import to_categorical # Multi-class labels converted to one-hot encoded categories

/usr/local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
from keras.preprocessing import sequence # Keras input preprocessing
from keras.models import Sequential # Keras model types
from keras.layers import Dense, Dropout, Activation # Keras model layers
from keras.layers import Embedding, LSTM
from keras.layers import Input, Dense, concatenate, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers import BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam
from keras.models import Model

In [3]:
def lstm_fcn_block():
    ip = Input(shape = (20, 300))
    
    # Recurrent side
    x = LSTM(8)(ip)
    x = Dropout(0.2)(x)

    # Fully Convolutional Side
    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding = "same", kernel_initializer = "he_uniform")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    y = Conv1D(256, 5, padding = "same", kernel_initializer = "he_uniform")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    y = Conv1D(128, 3, padding = "same", kernel_initializer = "he_uniform")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    y = GlobalAveragePooling1D()(y)

    # Merge both sides back together
    x = concatenate([x, y])
    
    # 2 output classes over softmax
    out = Dense(2, activation = "softmax")(x)

    model = Model(ip, out)

    model.summary()

    return model

In [4]:
# Load data from CSV
training_data = pd.read_csv('train_signal.csv')

In [5]:
# Peek at the data
training_data.head()

,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,N,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,N,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,N,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
3,B00003,~,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,~,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207


In [6]:
# Make a duplicate of the training_data for use in 1st layer
training_data_l1 = training_data.copy()

In [7]:
# Change to 2 classes for 2 layer classification

# Other & Noisy together
training_data_l1.loc[training_data_l1['Type'] == 'O', 'Type'] = 0
training_data_l1.loc[training_data_l1['Type'] == 'N', 'Type'] = 0

# AF and noisy together
training_data_l1.loc[training_data_l1['Type'] == 'A', 'Type'] = 1
training_data_l1.loc[training_data_l1['Type'] == '~', 'Type'] = 1

In [8]:
# Peek at the data
training_data_l1.head()

,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,0,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,0,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,0,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
3,B00003,1,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,1,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207


In [9]:
# Split into labels (y) and input (X)

# Data from the 3rd feature column onwards are input
X = training_data_l1.values[:,2:]
# Classes/Labels are the type of AF
y = training_data_l1["Type"].values
# Label is a binary integer
y=y.astype('int')

In [10]:
# Deal with missing data

# Replace missing values with Nan
X[X == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

In [11]:
print("X shape: " + str(X.shape))
print("y shape: " + str(y.shape))

X shape: (13062, 6000)
y shape: (13062,)


In [12]:
# One-hot encode class labels
print(y.shape)
y = to_categorical(y)
print(y.shape)

(13062,)
(13062, 2)


In [13]:
# Reshape input as 20 seconds of recording (for recurrent network)
X_3d = X.reshape((13062, 20, 300))


In [14]:
print("X shape: " + str(X_3d.shape))
print("y shape: " + str(y.shape))

X shape: (13062, 20, 300)
y shape: (13062, 2)


In [15]:
# Build layer 1 classifier
model_l1 = lstm_fcn_block()
# Tell model what loss function & optimiser to use
model_l1.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.01),
              metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 300)      0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 300, 20)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 300, 128)     20608       permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 300, 128)     512         conv1d_1[0][0]                   
____________________________________________________________________________________________

In [16]:
# train the model
model_l1.fit(X_3d, y, epochs=15, batch_size=32, validation_split=1/6)

Train on 10885 samples, validate on 2177 samples
Epoch 1/15
10885/10885 [==============================] - 65s 6ms/step - loss: 0.3617 - accuracy: 0.8876 - auc: 0.8846 - precision: 0.8792 - recall: 0.8792 - val_loss: 0.3731 - val_accuracy: 0.8755 - val_auc: 0.8942 - val_precision: 0.8856 - val_recall: 0.8856
Epoch 2/15
10885/10885 [==============================] - 65s 6ms/step - loss: 0.3449 - accuracy: 0.8881 - auc: 0.8980 - precision: 0.8858 - recall: 0.8858 - val_loss: 0.3727 - val_accuracy: 0.8755 - val_auc: 0.8996 - val_precision: 0.8858 - val_recall: 0.8858
Epoch 3/15
10885/10885 [==============================] - 64s 6ms/step - loss: 0.3353 - accuracy: 0.8889 - auc: 0.9023 - precision: 0.8854 - recall: 0.8854 - val_loss: 0.3789 - val_accuracy: 0.8755 - val_auc: 0.9046 - val_precision: 0.8861 - val_recall: 0.8861
Epoch 4/15
10885/10885 [==============================] - 64s 6ms/step - loss: 0.3316 - accuracy: 0.8878 - auc: 0.9069 - precision: 0.8865 - recall: 0.8865 - val_loss: 

In [17]:
predictions_from_first_layer = model_l1.predict(X_3d)
print(predictions_from_first_layer)

[[0.96460897 0.03539106]
 [0.96638197 0.03361809]
 [0.9613774  0.03862257]
 ...
 [0.9467135  0.05328647]
 [0.9835247  0.01647536]
 [0.9129899  0.08701008]]


In [18]:
# Check shape is correct
print(training_data.shape)
print(len(predictions_from_first_layer))

(13062, 6002)
13062


In [19]:
# Split into two datasets for 2nd layer classification
training_data_l2_NaO = training_data.loc[(training_data['Type'] == 'N') | (training_data['Type'] == 'O')]
training_data_l2_AaN = training_data.loc[(training_data['Type'] == 'A') | (training_data['Type'] == '~')]

print(training_data_l2_NaO.shape)
print(training_data_l2_AaN.shape)
training_data.head()

(11578, 6002)
(1484, 6002)


,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,N,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,N,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,N,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
3,B00003,~,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,~,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207


In [20]:
# Binary class encoding
training_data_l2_NaO.loc[training_data_l2_NaO['Type'] == 'O', 'Type'] = 0
training_data_l2_NaO.loc[training_data_l2_NaO['Type'] == 'N', 'Type'] = 1
training_data_l2_NaO.head()

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,1,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,1,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,1,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
7,B00007,0,-0.280,-0.481,-0.546,-0.594,-0.651,-0.659,-0.634,-0.626,...,-0.046,-0.022,-0.006,0.010,0.018,0.018,0.026,0.026,0.034,0.034
8,B00008,0,-0.393,-0.336,-0.288,-0.256,-0.224,-0.191,-0.159,-0.135,...,-0.280,-0.256,-0.232,-0.207,-0.183,-0.151,-0.127,-0.103,-0.087,-0.078


In [21]:
# Split into labels (y) and input (X)

# Data from the 3rd feature column onwards are input
X = training_data_l2_NaO.values[:,2:]
# Classes/Labels are the type of AF
y = training_data_l2_NaO["Type"].values
# Label is a binary integer
y=y.astype('int')

In [22]:
# Deal with missing data

# Replace missing values with Nan
X[X == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

In [23]:
# One-hot encode class labels
print(y.shape)
y = to_categorical(y)
print(y.shape)

(11578,)
(11578, 2)


In [24]:
# Reshape input as 20 seconds of recording (for recurrent network)
X_3d = X.reshape((11578, 20, 300))

In [25]:
print("X shape: " + str(X_3d.shape))
print("y shape: " + str(y.shape))

X shape: (11578, 20, 300)
y shape: (11578, 2)


In [26]:
# Build layer 2 Normal & Other classifier
model_l2_NaO = lstm_fcn_block()
# Tell model what loss function & optimiser to use
model_l2_NaO.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.01),
              metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20, 300)      0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 300, 20)      0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 300, 128)     20608       permute_2[0][0]                  
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 300, 128)     512         conv1d_4[0][0]                   
____________________________________________________________________________________________

In [27]:
# train the model
model_l2_NaO.fit(X_3d, y, epochs=15, batch_size=32, validation_split=1/6)

Train on 9648 samples, validate on 1930 samples
Epoch 1/15
9648/9648 [==============================] - 58s 6ms/step - loss: 0.6479 - accuracy: 0.6545 - auc_1: 0.6684 - precision_1: 0.6392 - recall_1: 0.6392 - val_loss: 0.6080 - val_accuracy: 0.6850 - val_auc_1: 0.6879 - val_precision_1: 0.6581 - val_recall_1: 0.6581
Epoch 2/15
9648/9648 [==============================] - 58s 6ms/step - loss: 0.6233 - accuracy: 0.6652 - auc_1: 0.6976 - precision_1: 0.6613 - recall_1: 0.6613 - val_loss: 0.6272 - val_accuracy: 0.6544 - val_auc_1: 0.7012 - val_precision_1: 0.6621 - val_recall_1: 0.6621
Epoch 3/15
9648/9648 [==============================] - 57s 6ms/step - loss: 0.6140 - accuracy: 0.6731 - auc_1: 0.7059 - precision_1: 0.6640 - recall_1: 0.6640 - val_loss: 0.6104 - val_accuracy: 0.6870 - val_auc_1: 0.7092 - val_precision_1: 0.6656 - val_recall_1: 0.6656
Epoch 4/15
9648/9648 [==============================] - 57s 6ms/step - loss: 0.6079 - accuracy: 0.6768 - auc_1: 0.7135 - precision_1: 0.668

In [28]:
# Binary class encoding
training_data_l2_AaN.loc[training_data_l2_AaN['Type'] == '~', 'Type'] = 0
training_data_l2_AaN.loc[training_data_l2_AaN['Type'] == 'A', 'Type'] = 1
training_data_l2_AaN.head()

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
3,B00003,0,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,0,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207
5,B00005,0,0.539,0.655,0.800,0.964,1.157,1.349,1.542,1.755,...,-1.708,-1.650,-1.563,-1.457,-1.351,-1.255,-1.158,-1.100,-1.033,-0.985
6,B00006,0,-0.927,-0.869,-0.782,-0.657,-0.473,-0.348,-0.261,-0.203,...,-1.409,-1.274,-1.120,-0.927,-0.657,-0.493,-0.396,-0.338,-0.309,-0.338
9,B00009,0,-0.130,-0.211,-0.308,-0.421,-0.531,-0.638,-0.748,-0.861,...,1.419,1.199,1.044,0.992,0.960,0.944,0.908,0.850,0.775,0.701


In [29]:
# Split into labels (y) and input (X)

# Data from the 3rd feature column onwards are input
X = training_data_l2_AaN.values[:,2:]
# Classes/Labels are the type of AF
y = training_data_l2_AaN["Type"].values
# Label is a binary integer
y=y.astype('int')

In [30]:
# Deal with missing data

# Replace missing values with Nan
X[X == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

In [31]:
# One-hot encode class labels
print(y.shape)
y = to_categorical(y)
print(y.shape)

(1484,)
(1484, 2)


In [32]:
# Reshape input as 20 seconds of recording (for recurrent network)
X_3d = X.reshape((1484, 20, 300))

In [33]:
print("X shape: " + str(X_3d.shape))
print("y shape: " + str(y.shape))

X shape: (1484, 20, 300)
y shape: (1484, 2)


In [34]:
# Build layer 2 AF & Noisy classifier
model_l2_AaN = lstm_fcn_block()
# Tell model what loss function & optimiser to use
model_l2_AaN.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.01),
              metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 20, 300)      0                                            
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 300, 20)      0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 300, 128)     20608       permute_3[0][0]                  
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 300, 128)     512         conv1d_7[0][0]                   
____________________________________________________________________________________________

In [35]:
# train the model
model_l2_AaN.fit(X_3d, y, epochs=15, batch_size=32, validation_split=1/6)

Train on 1236 samples, validate on 248 samples
Epoch 1/15
1236/1236 [==============================] - 9s 7ms/step - loss: 0.5467 - accuracy: 0.7759 - auc_2: 0.8008 - precision_2: 0.7641 - recall_2: 0.7641 - val_loss: 2.1227 - val_accuracy: 0.7863 - val_auc_2: 0.8126 - val_precision_2: 0.7760 - val_recall_2: 0.7760
Epoch 2/15
1236/1236 [==============================] - 7s 6ms/step - loss: 0.3908 - accuracy: 0.8301 - auc_2: 0.8386 - precision_2: 0.7937 - recall_2: 0.7937 - val_loss: 0.4334 - val_accuracy: 0.8266 - val_auc_2: 0.8549 - val_precision_2: 0.8025 - val_recall_2: 0.8025
Epoch 3/15
1236/1236 [==============================] - 7s 6ms/step - loss: 0.3095 - accuracy: 0.8762 - auc_2: 0.8723 - precision_2: 0.8178 - recall_2: 0.8178 - val_loss: 0.3950 - val_accuracy: 0.8306 - val_auc_2: 0.8824 - val_precision_2: 0.8248 - val_recall_2: 0.8248
Epoch 4/15
1236/1236 [==============================] - 7s 6ms/step - loss: 0.2723 - accuracy: 0.8811 - auc_2: 0.8938 - precision_2: 0.8329 - r

In [28]:
# Load in test data

# Run through layer 1 classifier (rfc)

# Run resulting class label 0s through to rfc_l2_NaO

#Run resulting class label 1s through to rfc_l2_AaN

In [63]:
# load test data
test_data = pd.read_csv('test_signal.csv')
# Peek at the data
test_data.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,C00000,-0.169,-0.174,-0.184,-0.189,-0.200,-0.210,-0.221,-0.226,-0.226,...,0.375,0.386,0.386,0.360,0.323,0.282,0.240,0.184,0.132,0.090
1,C00001,-0.184,-0.174,-0.169,-0.164,-0.158,-0.158,-0.158,-0.153,-0.153,...,-0.086,-0.117,-0.143,-0.148,-0.153,-0.153,-0.153,-0.153,-0.153,-0.158
2,C00002,1.050,1.622,2.143,2.552,2.653,2.675,2.614,2.249,1.734,...,-1.342,-1.354,-1.365,-1.370,-1.376,-1.370,-1.365,-1.354,-1.342,-1.326
3,C00003,2.259,2.667,3.046,3.376,3.583,3.579,3.334,2.872,2.514,...,-0.119,-0.119,-0.119,-0.116,-0.114,-0.109,-0.104,-0.097,-0.090,-0.083
4,C00004,-0.203,-0.203,-0.201,-0.201,-0.201,-0.199,-0.194,-0.189,-0.185,...,-0.008,0.020,0.039,0.044,0.046,0.046,0.049,0.049,0.051,0.051


In [64]:


# Data from the 2nd feature column onwards are input
test_X = test_data.values[:,1:]
test_ids = test_data['ID']
print(test_X.shape)
print(test_ids.shape)


(4000, 6000)
(4000,)


In [65]:
# Deal with missing data

# Replace missing values with Nan
test_X[test_X == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
test_X = imputer.fit_transform(test_X)

In [66]:
# Reshape input as 20 seconds of recording (for recurrent network)
test_X_3d = test_X.reshape((4000, 20, 300))
print(test_X_3d.shape)

(4000, 20, 300)


In [67]:
# Pass through 1st layer classifier
predictions_ontest_first_layer_softmax = model_l1.predict(test_X_3d)
print(predictions_ontest_first_layer_softmax)

[[0.95503104 0.04496894]
 [0.93077195 0.0692281 ]
 [0.71224684 0.28775322]
 ...
 [0.8888699  0.11113009]
 [0.83275557 0.16724446]
 [0.82662356 0.17337649]]


In [68]:
# Decode softmax
predictions_ontest_first_layer = []
for no_prob, an_prob in predictions_ontest_first_layer_softmax:
    if no_prob > an_prob:
        predictions_ontest_first_layer.append(0) # Normal & Other
    else:
        predictions_ontest_first_layer.append(1) # AF & Noisy

print(predictions_ontest_first_layer)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [69]:
# Add predictions from the first layer NOT USED NOW WOULD DO THSI WHEN ACTUALLY TESTING IT
# As column in the orginal training data
test_data.insert(1, "layer_1_class", predictions_ontest_first_layer, True) 
test_data.head()

,ID,layer_1_class,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,C00000,0,-0.169,-0.174,-0.184,-0.189,-0.200,-0.210,-0.221,-0.226,...,0.375,0.386,0.386,0.360,0.323,0.282,0.240,0.184,0.132,0.090
1,C00001,0,-0.184,-0.174,-0.169,-0.164,-0.158,-0.158,-0.158,-0.153,...,-0.086,-0.117,-0.143,-0.148,-0.153,-0.153,-0.153,-0.153,-0.153,-0.158
2,C00002,0,1.050,1.622,2.143,2.552,2.653,2.675,2.614,2.249,...,-1.342,-1.354,-1.365,-1.370,-1.376,-1.370,-1.365,-1.354,-1.342,-1.326
3,C00003,0,2.259,2.667,3.046,3.376,3.583,3.579,3.334,2.872,...,-0.119,-0.119,-0.119,-0.116,-0.114,-0.109,-0.104,-0.097,-0.090,-0.083
4,C00004,0,-0.203,-0.203,-0.201,-0.201,-0.201,-0.199,-0.194,-0.189,...,-0.008,0.020,0.039,0.044,0.046,0.046,0.049,0.049,0.051,0.051


In [71]:
# Split data set by result class label
test_data_l2_NaO = test_data.loc[(test_data['layer_1_class'] == 0)]
test_data_l2_AaN = test_data.loc[(test_data['layer_1_class'] == 1)]

print(test_data_l2_NaO.shape)
print(test_data_l2_AaN.shape)

(3983, 6002)
(17, 6002)


In [72]:
# Input is after 1st and 2nd columns
test_X_l2_NaO = test_data_l2_NaO.values[:,2:]
#record IDs
test_ids_l2_NaO = test_data_l2_NaO['ID']
print(test_X_l2_NaO.shape)
print(test_ids_l2_NaO.shape)

# Input is after 1st and 2nd columns
test_X_l2_AaN = test_data_l2_AaN.values[:,2:]
#record IDs
test_ids_l2_AaN = test_data_l2_AaN['ID']
print(test_X_l2_AaN.shape)
print(test_ids_l2_AaN.shape)


(3983, 6000)
(3983,)
(17, 6000)
(17,)


In [74]:
# Deal with missing data

# Replace missing values with Nan
test_X_l2_NaO[test_X_l2_NaO == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
test_X_l2_NaO = imputer.fit_transform(test_X_l2_NaO)

# Replace missing values with Nan
test_X_l2_AaN[test_X_l2_AaN == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
test_X_l2_AaN = imputer.fit_transform(test_X_l2_AaN)

In [75]:
# Reshape input as 20 seconds of recording (for recurrent network)
test_X_3d_l2_NaO = test_X_l2_NaO.reshape((3983, 20, 300))
print(test_X_3d_l2_NaO.shape)

test_X_3d_l2_AaN = test_X_l2_AaN.reshape((17, 20, 300))
print(test_X_3d_l2_AaN.shape)

(3983, 20, 300)
(17, 20, 300)


In [77]:
# Pass first subset through 2nd layer classifiers
predictions_ontest_NaO_2nd_layer_softmax = model_l2_NaO.predict(test_X_3d_l2_NaO)
print(predictions_ontest_NaO_2nd_layer_softmax)

[[0.49217194 0.50782806]
 [0.45131597 0.548684  ]
 [0.14931075 0.8506893 ]
 ...
 [0.2671297  0.73287034]
 [0.23066898 0.76933104]
 [0.13461745 0.86538255]]


In [78]:
# Pass second subset through 2nd layer classifiers
predictions_ontest_AaN_2nd_layer_softmax = model_l2_AaN.predict(test_X_3d_l2_AaN)
print(predictions_ontest_AaN_2nd_layer_softmax)

[[5.0999278e-01 4.9000716e-01]
 [1.8720317e-03 9.9812800e-01]
 [6.1984356e-02 9.3801564e-01]
 [7.0587546e-01 2.9412448e-01]
 [5.1640192e-05 9.9994838e-01]
 [2.9437733e-01 7.0562273e-01]
 [4.4952169e-02 9.5504779e-01]
 [1.9351978e-02 9.8064798e-01]
 [8.9831448e-01 1.0168549e-01]
 [2.2587297e-03 9.9774134e-01]
 [2.9675793e-03 9.9703240e-01]
 [3.6111072e-01 6.3888931e-01]
 [8.8024430e-02 9.1197562e-01]
 [4.7244900e-01 5.2755100e-01]
 [5.2208509e-02 9.4779152e-01]
 [2.4226280e-03 9.9757737e-01]
 [9.4154787e-01 5.8452085e-02]]


In [86]:
#Decode first subset's softmax
predictions_ontest_NaO_2nd_layer = []
for o_prob, n_prob in predictions_ontest_NaO_2nd_layer_softmax:
    if o_prob > n_prob:
        predictions_ontest_NaO_2nd_layer.append(0) # Other
    else:
        predictions_ontest_NaO_2nd_layer.append(1) # Normal

print(predictions_ontest_NaO_2nd_layer)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 

In [87]:
#Decode second subset's softmax
predictions_ontest_AaN_2nd_layer = []
for n_prob, a_prob in predictions_ontest_AaN_2nd_layer_softmax:
    if n_prob > a_prob:
        predictions_ontest_AaN_2nd_layer.append(0) # Noise
    else:
        predictions_ontest_AaN_2nd_layer.append(1) # Af

print(predictions_ontest_AaN_2nd_layer)

[0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]


In [88]:
#Merge predictions with IDs

NaO_preds = list(zip(test_ids_l2_NaO, predictions_ontest_NaO_2nd_layer))
print(NaO_preds)

AaN_preds = list(zip(test_ids_l2_AaN, predictions_ontest_AaN_2nd_layer))
print(AaN_preds)



[('C00000', 1), ('C00001', 1), ('C00002', 1), ('C00003', 1), ('C00004', 1), ('C00005', 1), ('C00006', 1), ('C00007', 1), ('C00008', 1), ('C00009', 1), ('C00010', 1), ('C00011', 1), ('C00012', 1), ('C00013', 1), ('C00014', 1), ('C00015', 1), ('C00016', 1), ('C00017', 1), ('C00018', 1), ('C00019', 1), ('C00020', 1), ('C00021', 0), ('C00022', 0), ('C00023', 1), ('C00024', 1), ('C00025', 1), ('C00026', 1), ('C00027', 1), ('C00028', 1), ('C00029', 1), ('C00030', 1), ('C00031', 1), ('C00032', 1), ('C00033', 1), ('C00034', 1), ('C00035', 1), ('C00036', 1), ('C00037', 1), ('C00038', 1), ('C00039', 1), ('C00040', 1), ('C00041', 1), ('C00042', 1), ('C00043', 0), ('C00044', 0), ('C00045', 1), ('C00046', 1), ('C00047', 1), ('C00049', 1), ('C00050', 1), ('C00051', 1), ('C00052', 1), ('C00053', 1), ('C00054', 1), ('C00055', 1), ('C00056', 1), ('C00057', 1), ('C00058', 1), ('C00059', 1), ('C00060', 1), ('C00061', 1), ('C00062', 1), ('C00063', 1), ('C00064', 1), ('C00065', 1), ('C00066', 1), ('C00067'

In [89]:
# Get Normal & Other predictions
predictions_NaO = pd.DataFrame(NaO_preds, columns=['ID', 'Encoding'])
predictions_NaO.head()

def decode_NaO(row):
     # 'O'==0, 'N'==1
    if row['Encoding'] == 0:
        return 'O'
    else:
        return 'N'

predictions_NaO['Predicted'] = predictions_NaO.apply(decode_NaO, axis=1)
predictions_NaO.head()
   

,ID,Encoding,Predicted
0,C00000,1,N
1,C00001,1,N
2,C00002,1,N
3,C00003,1,N
4,C00004,1,N


In [90]:
# Get AF & Noisy predictions
predictions_AaN = pd.DataFrame(AaN_preds, columns=['ID', 'Encoding'])
predictions_AaN.head()

def decode_AaN(row):
    # 'A'==1, '~'==0
    if row['Encoding'] == 0:
        return '~'
    else:
        return 'A'
    
predictions_AaN['Predicted'] = predictions_AaN.apply(decode_AaN, axis=1)
predictions_AaN.head()

,ID,Encoding,Predicted
0,C00048,0,~
1,C00239,1,A
2,C00605,1,A
3,C00796,0,~
4,C00810,1,A


In [42]:
# # Randonly guess the rows that were removed because of missing values
# removed_rows_predictions = pd.DataFrame(removed_rows, columns=['ID'])
# guesses1 = [1]*removed_rows_predictions.shape[0]
# removed_rows_predictions['Encoding'] = guesses1
# guesse2 = ['A']*removed_rows_predictions.shape[0]
# removed_rows_predictions['Predicted'] = guesse2


# removed_rows_predictions.head()

,ID,Encoding,Predicted
1600,C01600,1,A


In [92]:
# Merge Dataframes into one
frames = [predictions_NaO, predictions_AaN]
predictions = pd.concat(frames)
print(predictions.shape)
predictions.head()

(4000, 3)


,ID,Encoding,Predicted
0,C00000,1,N
1,C00001,1,N
2,C00002,1,N
3,C00003,1,N
4,C00004,1,N


In [93]:
# Ensure Sorted by ID
predictions.sort_values(by=['ID'], inplace=True)
print(predictions)

          ID  Encoding Predicted
0     C00000         1         N
1     C00001         1         N
2     C00002         1         N
3     C00003         1         N
4     C00004         1         N
...      ...       ...       ...
3978  C03995         1         N
3979  C03996         1         N
3980  C03997         1         N
3981  C03998         1         N
3982  C03999         1         N

[4000 rows x 3 columns]


In [94]:
# We have dropped all rows with missing values 
# Therefore have 3999 rows instead of the original 4000
# WHAT TO DO WHEN FOR SUBMISSION??

In [95]:
# Drop endoding column
results = pd.DataFrame(predictions, columns=['ID', 'Predicted'])
# Write to file
results.to_csv("submission_LSTM_FCN_2_layer_framework.csv", index=False)